# Prepare your ratings data

The following case-study demonstrates how to prepare your ratings for further
processing.

## Preliminary tasks
As a first step, we are going to import a portfolio into a `pd.DataFrame`. We'll call
it `port_df`. This dataframe comprises a number of securities with respective weights
 and ratings from different rating agencies.

In [ ]:
import pandas as pd

import pyratings as rtg

port_df = pd.read_excel("portfolio.xlsx")
port_df.head()

## Clean your ratings
By looking at the very first rows of `port_df`, we can see that some ratings do have a
_watch_ attached. This is marked by the `*-` suffix, which follows the actual rating
after a blank. Other ratings are unsolicited. In this case, the rating is followed by
the letter `u`.

Before we can use these ratings for further computations, we need to
[clean](../../user_guide/cleansing/) the ratings and get rid of everything that is not
part of the actual rating. That is, we need to strip off _watches_ etc.

We are going to use the
[get_pure_ratings](../../api_reference/#pyratings.clean.get_pure_ratings) function.
This function works on strings, so we need to make sure to pass the relevant columns.


In [ ]:
ratings_clean_df = rtg.get_pure_ratings(
    port_df.loc[:, ["BB Comp", "SP", "Moody", "Fitch"]]
)
ratings_clean_df.head()

As you can see, the suffix "\_clean" has automatically been added to the column
headings. Let's add these _clean_ ratings to `port_df` by simple concatenation.

In [ ]:
port_df = pd.concat([port_df, ratings_clean_df], axis=1)
port_df.head()

## Consolidate your ratings
The natural next step is to [consolidate](../../user_guide/consolidate/) the ratings;
i.e. we need to assign one and only one concrete rating to any individual security.

Let's compute the _worst ratings_ and directly concatenate the newly computed
column to our existing `port_df` in one batch. **pyratings** automatically names the
new column "worst_rtg".

We are going to use
[``get_worst_ratings``](../../api_reference/#pyratings.consolidate.get_worst_ratings)
 and use S&P's rating scale to display the data.

In [ ]:
port_df = pd.concat(
    [
        port_df,
        rtg.get_worst_ratings(
            ratings=ratings_clean_df,
            rating_provider_input=["Bloomberg", "S&P", "Moody's", "Fitch"],
            rating_provider_output="S&P"
        )
    ],
    axis=1,
)

port_df.head()

There you are. You have all your securities and only one concrete rating attached to
each one (in this case the __worst rating__).

In [ ]:
port_df.loc[:, ["ISIN", "weight", "worst_rtg"]].head()